### build vocab file and calculate appearence-fraction of words

In [1]:
import nltk

def get_frac_dist(token_list):
    '''
    Computes frequency count and fraction of individual words in a list.

    Parameters
    ----------
    token_list : list
        List of all (non-unique) tokens from some corpus

    Returns
    -------
    dict
        Dictionary of { token, (count, fraction) } pairs.
    '''

    token_list = nltk.word_tokenize(content)
    total_token_count = len(token_list)

    freq_dict = nltk.FreqDist(token_list)

    frac_dict = {}
    for token, count in freq_dict.items():
        frac_dict[token] = (count, count / total_token_count)

    return frac_dict

In [2]:
with open("C:/Users/flori/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20NG/idocnade/vocabbuilder.txt") as f:
    content = f.read().lower()

token_list = nltk.word_tokenize(content.lower())
frac_dist = get_frac_dist(token_list)

In [3]:
i = 0
for k, v in frac_dist.items():

    i += 1
    if i > 10:
        break

    token = k.lower()
    count = v[0]
    frac = v[1]
    print('%-20s: %5d, %0.5f' % (token, count, frac))

car                 :  3460, 0.00113
nntppostinghost     :  9462, 0.00310
rac                 :    24, 0.00001
organization        : 22710, 0.00744
university          : 11136, 0.00365
maryland            :   248, 0.00008
college             :  1254, 0.00041
park                :   498, 0.00016
lines               : 18296, 0.00599
wonder              :  1318, 0.00043


In [4]:
l = [ (k, v[0], v[1]) for k, v in frac_dist.items() if v[0] > 100]
print(len(l))

4119


In [5]:
len(frac_dist)

108202

In [6]:
print(l[0][0])

car


In [7]:
vocab = set()
i = 0
for i in range(len(l)):
    #print(word)
    vocab.add(l[i][0])
vocab = sorted(vocab)
print(len(vocab))

4119


In [8]:
vocab[1]

'aa'

In [9]:
with open("C:/Users/flori/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20NG/idocnade/vocab_docnade.vocab", "w") as f:
    for item in vocab:
        f.write(item + "\n")
f.close()  

In [10]:
import pandas as pd

datatrain = pd.read_csv('C:/Users/flori/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20NG/processed/training.csv') 
datatest = pd.read_csv('C:/Users/flori/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20NG/processed/test.csv') 

In [11]:
import spacy
import nltk
from nltk.tokenize import word_tokenize

In [12]:
# remove stopwords and only keep words that are in vocab

def remove_stopwords(text):
    text_tokens = word_tokenize(text)
    tokens_without_sw= [word for word in text_tokens if not word in all_stopwords and word in vocab]
    return tokens_without_sw

sp = spacy.load('en_core_web_lg')
all_stopwords = sp.Defaults.stop_words


datatrain["text"] = datatrain['text'].apply(remove_stopwords)
datatest["text"] = datatest['text'].apply(remove_stopwords)
datatrain['length']=datatrain['text'].apply(len)
datatest['length']=datatest['text'].apply(len)


In [13]:
label_dict = {
    "0": 'alt.atheism',
     "1":'comp.graphics',
     "2":'comp.os.ms-windows.misc',
     "3":'comp.sys.ibm.pc.hardware',
     "4":'comp.sys.mac.hardware',
     "5":'comp.windows.x',
     "6":'misc.forsale',
     "7":'rec.autos',
     "8":'rec.motorcycles',
     "9":'rec.sport.baseball',
     "10":'rec.sport.hockey',
     "11":'sci.crypt',
     "12":'sci.electronics',
     "13":'sci.med',
     "14":'sci.space',
     "15":'soc.religion.christian',
     "16":'talk.politics.guns',
     "17":'talk.politics.mideast',
     "18":'talk.politics.misc',
     "19":'talk.religion.misc'
}

In [14]:
datatrain["label"]=""
for index, row in datatrain.iterrows():
    datatrain["label"][index] = label_dict[str(row["target"])]
datatest["label"]=""
for index, row in datatest.iterrows():
    datatest["label"][index] = label_dict[str(row["target"])]    
    

datatrain.head()

<ipython-input-14-3591ac13fd2c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datatrain["label"][index] = label_dict[str(row["target"])]
<ipython-input-14-3591ac13fd2c>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datatest["label"][index] = label_dict[str(row["target"])]


,text,target,length,label
0,"[car, wonder, car, day, door, sport, car, look...",7,33,rec.autos
1,"[final, clock, report, fair, number, soul, upg...",4,46,comp.sys.mac.hardware
2,"[question, usa, folk, mac, plus, finally, ghos...",4,112,comp.sys.mac.hardware
3,"[world, version, write, write, article, know, ...",1,27,comp.graphics
4,"[sci, article, article, write, clear, warn, me...",14,55,sci.space


In [15]:
frames = [datatrain,datatest]
data = pd.concat(frames)
data.head()

,text,target,length,label
0,"[car, wonder, car, day, door, sport, car, look...",7,33,rec.autos
1,"[final, clock, report, fair, number, soul, upg...",4,46,comp.sys.mac.hardware
2,"[question, usa, folk, mac, plus, finally, ghos...",4,112,comp.sys.mac.hardware
3,"[world, version, write, write, article, know, ...",1,27,comp.graphics
4,"[sci, article, article, write, clear, warn, me...",14,55,sci.space


In [16]:
print(len(datatrain),len(datatest),len(data))

11314 7532 18846


In [17]:
## 20NGfile
f = open("C:/Users/flori/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20NG/idocnade/20NG.txt","w",encoding='utf-8')
for index,row in data.iterrows():
    if row["text"] == []:
        continue
    text = " ".join(row["text"])
    f.write(row["label"]+"\t"+text+"\n")
f.close()

In [18]:
# split for evaluation set
from sklearn.model_selection import train_test_split
train_data, val_data = train_test_split(datatrain, test_size=0.2, random_state=42)

In [19]:
## 20NG train txt
f = open("C:/Users/flori/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20NG/idocnade/20NG_train.txt","w",encoding='utf-8')
for index,row in train_data.iterrows():
    if row["text"] == []:
        continue
    text = " ".join(row["text"])
    f.write(row["label"]+"\t"+text+"\n")
f.close()



In [20]:
## 20NG val txt
f = open("C:/Users/flori/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20NG/idocnade/20NG_val.txt","w",encoding='utf-8')
for index,row in val_data.iterrows():
    if row["text"] == []:
        continue
    text = " ".join(row["text"])
    f.write(row["label"]+"\t"+text+"\n")
f.close()


In [21]:
## 20NG test txt
f = open("C:/Users/flori/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20NG/idocnade/20NG_test.txt","w",encoding='utf-8')
for index,row in datatest.iterrows():
    if row["text"] == []:
        continue
    text = " ".join(row["text"])
    f.write(row["label"]+"\t"+text+"\n")
f.close()


In [22]:
## 20NG text only  txt
f = open("C:/Users/flori/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20NG/idocnade/20NGTextonly.txt","w",encoding='utf-8')
for index,row in data.iterrows():
    if row["text"] == []:
        continue
    text = " ".join(row["text"])
    f.write(text+"\n")
f.close()

In [23]:
# test.csv
datatestcsv = datatest.copy()
for index,row in datatestcsv.iterrows():
    if row["text"] == []:
        continue
    datatestcsv["text"][index] = " ".join(row["text"])
datatestcsv.to_csv(path_or_buf="C:/Users/flori/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20NG/idocnade/test.csv",columns=["label","text"],header=False,index=False,encoding="utf-8",line_terminator="\n")


<ipython-input-23-80d1eedc2c26>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datatestcsv["text"][index] = " ".join(row["text"])


In [24]:
# training.csv
datatraincsv = train_data.copy()
for index,row in datatraincsv.iterrows():
    if row["text"] == []:
        continue
    datatraincsv["text"][index] = " ".join(row["text"])
datatraincsv.to_csv(path_or_buf="C:/Users/flori/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20NG/idocnade/training.csv",columns=["label","text"],header=False,index=False,encoding="utf-8",line_terminator="\n")


<ipython-input-24-b4b80c005669>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datatraincsv["text"][index] = " ".join(row["text"])


In [25]:
# validation.csv
datavalcsv = val_data.copy()
for index,row in datavalcsv.iterrows():
    if row["text"] == []:
        continue
    datavalcsv["text"][index] = " ".join(row["text"])
datavalcsv.to_csv(path_or_buf="C:/Users/flori/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20NG/idocnade/validation.csv",columns=["label","text"],header=False,index=False,encoding="utf-8",line_terminator="\n")


<ipython-input-25-8c3ae67525e7>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datavalcsv["text"][index] = " ".join(row["text"])


In [26]:
print(len(datatrain),len(datatest),len(data))

11314 7532 18846


In [27]:
word2id = dict()
i = 0
for word in vocab:
    word2id[word] = i
    i+=1

In [28]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train')
label_mapping = dict()
i = 0
for word in newsgroups_train.target_names:
    label_mapping[word]=i
    i+=1

In [29]:
#test_docnade.xlsx
with open("C:/Users/flori/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20NG/idocnade/20NG_test.txt", "r") as f:
    content = f.readlines()
content = [x.strip() for x in content] 
content= [x.split("\t") for x in content]
csvfile = pd.DataFrame(content,columns=["label","text"])
for index,row in csvfile.iterrows():
    id_string = str()
    for word in row["text"].split():
        id_string += " "+str(word2id[word])
    label_id = label_mapping[row["label"]]    
    csvfile["text"][index] = id_string
    csvfile["label"][index] = label_id
csvfile.to_csv(path_or_buf="C:/Users/flori/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20NG/idocnade/test_docnade.csv",columns=["label","text"],header=False,index=False,encoding="utf-8",line_terminator="\n")    

In [30]:
#training_docnade.xlsx
with open("C:/Users/flori/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20NG/idocnade/20NG_train.txt", "r") as f:
    content = f.readlines()
content = [x.strip() for x in content] 
content= [x.split("\t") for x in content]
csvfile = pd.DataFrame(content,columns=["label","text"])

for index,row in csvfile.iterrows():
    id_string = str()
    for word in row["text"].split():
        id_string += " "+str(word2id[word])
    label_id = label_mapping[row["label"]]    
    csvfile["text"][index] = id_string
    csvfile["label"][index] = label_id
csvfile.to_csv(path_or_buf="C:/Users/flori/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20NG/idocnade/training_docnade.csv",columns=["label","text"],header=False,index=False,encoding="utf-8",line_terminator="\n")

In [31]:
#validation_docnade.xlsx
with open("C:/Users/flori/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20NG/idocnade/20NG_val.txt", "r") as f:
    content = f.readlines()
content = [x.strip() for x in content] 
content= [x.split("\t") for x in content]
csvfile = pd.DataFrame(content,columns=["label","text"])
csvfile.head()
for index,row in csvfile.iterrows():
    id_string = str()
    for word in row["text"].split():
        id_string += " "+str(word2id[word])
    label_id = label_mapping[row["label"]]    
    csvfile["text"][index] = id_string
    csvfile["label"][index] = label_id
csvfile.to_csv(path_or_buf="C:/Users/flori/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20NG/idocnade/validation_docnade.csv",columns=["label","text"],header=False,index=False,encoding="utf-8",line_terminator="\n")